# Mask R-CNN for Keypoint Detection

Example showing how to do keypoint detection with Mask R-CNN

In [1]:
USE_GPU = False
TRAIN   = True

%matplotlib inline
import importlib
import os

if not USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import imgaug

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

c:\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load COCO Datasets for Keypoint Detection

In [2]:
from coco import coco_keypoints
importlib.reload(coco_keypoints)

# COCO dataset dir
COCO_DATA_DIR = "A:/Programming/DeepLearningDatasets/coco" if os.path.isdir("A:/Programming/DeepLearningDatasets/coco") else os.path.join(ROOT_DIR, "data/coco")

# Prepare dataset
#keypoints_to_include = ['nose', 'left_shoulder', 'right_shoulder',
#                        'left_elbow', 'right_elbow', 'left_wrist',
#                        'right_wrist', 'left_hip', 'right_hip',
#                        'left_knee', 'right_knee', 'left_ankle',
#                        'right_ankle']
keypoints_to_include = ["nose", "right_wrist"]

dataset_train = coco_keypoints.CocoDataset()
dataset_train.load_coco(COCO_DATA_DIR, subset="train", year="2017", auto_download=True, keypoints_to_include=keypoints_to_include)
dataset_train.prepare()

dataset_val = coco_keypoints.CocoDataset()
dataset_val.load_coco(COCO_DATA_DIR, subset="val", year="2017", auto_download=True, keypoints_to_include=keypoints_to_include)
dataset_val.prepare()

Will use images in A:/Programming/DeepLearningDatasets/coco/train2017
Will use annotations in A:/Programming/DeepLearningDatasets/coco/annotations/person_keypoints_train2017.json
loading annotations into memory...
Done (t=9.85s)
creating index...
index created!
Will use images in A:/Programming/DeepLearningDatasets/coco/val2017
Will use annotations in A:/Programming/DeepLearningDatasets/coco/annotations/person_keypoints_val2017.json
loading annotations into memory...
Done (t=0.33s)
creating index...
index created!


### Setup Training

In [3]:
class TrainConfig(Config):
    NAME = "coco_keypoints"
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1 # IMAGES_PER_GPU = 2
    GPU_COUNT = 1
    
    # We use resnet50
    BACKBONE = "resnet50"
    
    # Set number of classes
    NUM_CLASSES = dataset_train.num_classes
    NUM_KEYPOINTS = dataset_train.num_kp_classes
    
    # DEBUG:
    STEPS_PER_EPOCH = 1000 if USE_GPU else 10
    
    TRAIN_ROIS_PER_IMAGE = 100
    #MAX_GT_INSTANCES = 1
    
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256

config = TrainConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
NAM

## Train

In [4]:
if TRAIN:
    importlib.reload(modellib)

    # Create model
    print("Creating model...")
    model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

    # Load weights trained on MS-COCO
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       #exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_mask"])

Creating model...

build:
input_gt_boxes.shape (?, ?, 4)
input_gt_kp_ids.shape (?, ?, 2)
input_gt_masks.shape (?, ?, 2)
gt_masks.shape (?, 2)
masks.shape (?, 2)
kps_1d_indices.shape (?,)
kps_x.shape (?, ?)
kps_x.shape (?, ?)
resized_kps_x.shape (?, ?)
kps_indices.shape (?, ?)
masks.shape (?, ?)
target_mask.shape (1, ?, ?)
target_mask.dtype <dtype: 'int32'>
mrcnn_mask.shape (?, 100, 2, 28, 28)

mrcnn_mask_loss_graph shapes:
target_masks.shape (1, ?, ?)
target_masks.dtype <dtype: 'int32'>
target_kp_ids.shape (1, ?, ?)
pred_masks.shape (?, 100, 2, 28, 28)
y_true.shape (?,)
y_pred.shape (?, ?)


mrcnn_mask_loss_graph shapes:
target_masks.shape (?, 100, 2)
target_masks.dtype <dtype: 'int32'>
target_kp_ids.shape (?, 2)
pred_masks.shape (?, 100, 2, 28, 28)
y_true.shape (?,)
y_pred.shape (?, ?)



In [ ]:
if TRAIN:
    importlib.reload(modellib)

    # Training - Stage 1
    print("Training network heads...")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=4,
                layers="heads",
                augmentation=None)

Training network heads...

Starting at epoch 0. LR=0.001

Checkpoint Path: A:\Programming\Pose_RCNN\logs\coco_keypoints20180602T0058\mask_rcnn_coco_keypoints_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_

c:\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
10/10 [==============================] - 217s 22s/step - loss: 3.8899 - rpn_class_loss: 0.0627 - rpn_bbox_loss: 1.5025 - mrcnn_bbox_loss: 0.9776 - mrcnn_mask_loss: 1.3472 - val_loss: 4.8186 - val_rpn_class_loss: 0.0671 - val_rpn_bbox_loss: 0.8435 - val_mrcnn_bbox_loss: 0.9763 - val_mrcnn_mask_loss: 2.9316

In [ ]:
if TRAIN:
    importlib.reload(modellib)

    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up...")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=12,
                layers="4+",
                augmentation=None)

Fine tune Resnet stage 4 and up...

Starting at epoch 4. LR=0.001

Checkpoint Path: A:\Programming\Pose_RCNN\logs\coco_keypoints20180602T0058\mask_rcnn_coco_keypoints_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
res4d_branch2b         (Conv2D)
bn4d_branch2b          

c:\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 5/12
 9/10 [==========================>...] - ETA: 11s - loss: 3.6679 - rpn_class_loss: 0.0587 - rpn_bbox_loss: 0.7830 - mrcnn_bbox_loss: 0.6044 - mrcnn_mask_loss: 2.2218 

ERROR:root:Error processing image {'id': 421060, 'source': 'coco', 'path': 'A:/Programming/DeepLearningDatasets/coco/val2017\\000000421060.jpg', 'width': 640, 'height': 418, 'annotations': [{'segmentation': [[139.61, 259.01, 155.1, 252.27, 182.71, 209.84, 168.56, 216.58, 167.89, 204.45, 175.97, 197.05, 192.81, 181.56, 217.05, 172.8, 249.38, 163.37, 256.12, 149.9, 268.91, 149.9, 279.69, 157.98, 281.04, 165.39, 274.97, 177.51, 289.12, 194.35, 291.81, 222.64, 283.73, 250.92, 261.5, 272.47, 246.69, 267.09, 252.75, 254.29, 243.32, 261.03, 229.18, 262.37, 215.71, 248.9, 210.99, 261.7, 204.93, 271.8, 192.81, 275.17, 177.32, 263.72, 177.32, 277.19]], 'num_keypoints': 12, 'area': 11417.7497, 'iscrowd': 0, 'keypoints': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 243, 172, 2, 285, 202, 2, 203, 186, 2, 279, 238, 2, 180, 201, 2, 261, 258, 2, 219, 224, 2, 237, 244, 2, 190, 217, 2, 209, 240, 2, 163, 259, 2, 195, 261, 2], 'image_id': 421060, 'bbox': [139.61, 149.9, 152.2, 127.29], 'category_id': 1, 

10/10 [==============================] - 218s 22s/step - loss: 4.2212 - rpn_class_loss: 0.0593 - rpn_bbox_loss: 0.8788 - mrcnn_bbox_loss: 0.6141 - mrcnn_mask_loss: 2.6689 - val_loss: 4.1279 - val_rpn_class_loss: 0.0489 - val_rpn_bbox_loss: 0.7097 - val_mrcnn_bbox_loss: 0.7172 - val_mrcnn_mask_loss: 2.6521
Epoch 6/12
10/10 [==============================] - 212s 21s/step - loss: 5.6149 - rpn_class_loss: 0.0478 - rpn_bbox_loss: 0.9989 - mrcnn_bbox_loss: 0.5745 - mrcnn_mask_loss: 3.9937 - val_loss: 4.8698 - val_rpn_class_loss: 0.0752 - val_rpn_bbox_loss: 1.1967 - val_mrcnn_bbox_loss: 0.5270 - val_mrcnn_mask_loss: 3.0710

ERROR:root:Error processing image {'id': 272148, 'source': 'coco', 'path': 'A:/Programming/DeepLearningDatasets/coco/val2017\\000000272148.jpg', 'width': 640, 'height': 378, 'annotations': [{'segmentation': [[201.51, 256.72, 203.58, 256, 202.68, 254.11, 202.41, 252.32, 202.86, 251.24, 204.93, 250.43, 207.36, 250.97, 207.81, 253.49, 207.27, 255.55, 209.16, 255.73, 212.21, 258.25, 212.3, 259.51, 210.15, 261.31, 211.31, 262.75, 209.97, 263.02, 206.37, 264.27, 204.21, 261.76, 203.85, 260.77, 201.33, 264.18, 199.36, 263.47, 199.63, 261.67, 200.62, 260.5, 201.15, 259.51, 201.42, 257.98]], 'num_keypoints': 0, 'area': 99.6422, 'iscrowd': 0, 'keypoints': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'image_id': 272148, 'bbox': [199.36, 250.43, 12.94, 13.84], 'category_id': 1, 'id': 476226}, {'segmentation': [[289.42, 265.31, 289.8, 262.46, 292.59, 262.33, 293.88, 263.63, 294.01, 265.83, 

10/10 [==============================] - 222s 22s/step - loss: 6.4585 - rpn_class_loss: 0.0623 - rpn_bbox_loss: 1.1252 - mrcnn_bbox_loss: 0.5824 - mrcnn_mask_loss: 4.6886 - val_loss: 4.8008 - val_rpn_class_loss: 0.0704 - val_rpn_bbox_loss: 0.8516 - val_mrcnn_bbox_loss: 0.5473 - val_mrcnn_mask_loss: 3.3314
Epoch 8/12
 9/10 [==========================>...] - ETA: 11s - loss: 6.0418 - rpn_class_loss: 0.0752 - rpn_bbox_loss: 1.7038 - mrcnn_bbox_loss: 0.5959 - mrcnn_mask_loss: 3.6669 

ERROR:root:Error processing image {'id': 308587, 'source': 'coco', 'path': 'A:/Programming/DeepLearningDatasets/coco/val2017\\000000308587.jpg', 'width': 480, 'height': 640, 'annotations': [{'segmentation': [[114.29, 372.06, 113.25, 370.78, 114.67, 368.58, 115.83, 368.33, 116.48, 369.74, 117.12, 371.16, 116.09, 372.06, 116.35, 373.87, 119.44, 371.94, 120.34, 371.81, 120.6, 373.22, 118.8, 375.93, 116.22, 376.06, 116.99, 379.67, 120.34, 382.38, 120.22, 383.54, 117.89, 384.57, 113.13, 382.25, 112.35, 377.22, 113.13, 373.48]], 'num_keypoints': 0, 'area': 72.68655, 'iscrowd': 0, 'keypoints': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'image_id': 308587, 'bbox': [112.35, 368.33, 8.25, 16.24], 'category_id': 1, 'id': 485094}, {'segmentation': [[363.48, 342.29, 364.2, 344.07, 365.26, 344.3, 365.38, 353.67, 364.32, 353.32, 364.32, 351.42, 362.18, 348.93, 361.82, 347.63, 363.01, 341.93

10/10 [==============================] - 225s 23s/step - loss: 5.8203 - rpn_class_loss: 0.0808 - rpn_bbox_loss: 1.9028 - mrcnn_bbox_loss: 0.5363 - mrcnn_mask_loss: 3.3002 - val_loss: 4.8117 - val_rpn_class_loss: 0.0592 - val_rpn_bbox_loss: 1.0179 - val_mrcnn_bbox_loss: 0.5220 - val_mrcnn_mask_loss: 3.2125
Epoch 9/12
10/10 [==============================] - 216s 22s/step - loss: 3.6851 - rpn_class_loss: 0.0460 - rpn_bbox_loss: 0.5777 - mrcnn_bbox_loss: 0.4347 - mrcnn_mask_loss: 2.6268 - val_loss: 4.1583 - val_rpn_class_loss: 0.0583 - val_rpn_bbox_loss: 0.7520 - val_mrcnn_bbox_loss: 0.5478 - val_mrcnn_mask_loss: 2.8002

In [ ]:
if TRAIN:
    importlib.reload(modellib)

    # Training - Stage 3
    # Fine tune all layers
    print("Fine tune all layers...")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=16,
                layers="all",
                augmentation=None)
    
    # Training done
    print("Training done")

Explanation of losses:

- rpn_class_loss: "How well does the network separate positive from negative anchors?"
- rpn_bbox_loss: "How accurate are proposed the bounding boxes?"
- mrcnn_class_loss: "How well does the network distinguish people from background RoIs?" (really, this is the same as 1.)
- mrcnn_bbox_loss: "Bounding box refinement loss" (is this the same as 2.?)
- mrcnn_mask_loss: "How accurate are the predicted keypoints?" (per mask softmax cross-entropy loss)

In [ ]:
class InferenceConfig(coco_keypoints.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    BACKBONE = "resnet50"
    NUM_CLASSES = dataset_train.num_classes
    NUM_KEYPOINTS = dataset_train.num_kp_classes
    #MAX_GT_INSTANCES = 2
    
inference_config = InferenceConfig()
inference_config.display()

In [ ]:
importlib.reload(modellib)
importlib.reload(utils)

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

## Inspect Model

In [ ]:
importlib.reload(visualize)

# Load a test image
image_id = dataset_val.image_ids[10]

# Get ground truth masks and bboxes
image, image_meta, gt_class_ids, gt_bbox, gt_kp_masks, gt_kp_ids = \
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id)

log("image", image)
log("gt_class_ids", gt_class_ids)
log("gt_kp_ids", gt_kp_ids)
log("gt_bbox", gt_bbox)
log("gt_masks", gt_kp_masks)

visualize.display_keypoints(image, gt_bbox, gt_kp_masks, gt_kp_ids,
                            #skeleton=dataset_train.skeleton,
                            figsize=(8, 8))

In [ ]:
importlib.reload(modellib)
results = model.detect([image], verbose=1)
r = results[0]

In [ ]:
bbox, kp_masks, kp_ids = r["rois"], r["masks"], np.repeat(np.expand_dims(np.arange(17), axis=0), 12, axis=0)

log("image", image)
log("class_ids", r['class_ids'])
log("kp_ids", kp_ids)
log("bbox", bbox)
log("masks", kp_masks)

%matplotlib inline
import matplotlib.pyplot as plt

n = np.sqrt(inference_config.NUM_KEYPOINTS)
_, axes = plt.subplots(int(np.ceil(n)), int(np.floor(n)), figsize=(15, 15))
if n <= 1: axes = np.array(axes)
for i, ax in enumerate(axes.flatten()):
    ax.axis("off")
    if i >= inference_config.NUM_KEYPOINTS: continue
    #x, y = i % 4, i // 4
    
    ax.set_title(dataset_train.kp_class_names[i+1])
    ax.imshow(kp_masks[0, i])


plt.show()

visualize.display_keypoints(image, bbox[0:1], kp_masks[0:1],
                            #skeleton=dataset_train.skeleton
                            figsize=(15, 15))

In [ ]:
print(r['class_ids'])
print(r["scores"])

In [ ]:
#visualize.display_keypoints(original_image, r['rois'], r['masks'], r['class_ids'],
#                            dataset_val.class_names, skeleton=None)

In [ ]:
# Get predictions of mask head
mrcnn = model.run_graph([image], [
    ("detections", model.keras_model.get_layer("mrcnn_detection").output),
    ("masks", model.keras_model.get_layer("mrcnn_mask").output),
])

# Get detection class IDs. Trim zero padding.
det_class_ids = mrcnn['detections'][0, :, 4].astype(np.int32)
#print(det_class_ids)
#det_count = np.where(det_class_ids == 0)[0][0]
#det_class_ids = det_class_ids[:det_count]

#print("{} detections: {}".format(
#    det_count, np.array(dataset.class_names)[det_class_ids]))

In [ ]:
# Masks
"""det_boxes = utils.denorm_boxes(mrcnn["detections"][0, :, :4], image.shape[:2])
det_mask_specific = np.array([mrcnn["masks"][0, i, :, :, c] 
                              for i, c in enumerate(det_class_ids)])
det_masks = np.array([utils.unmold_mask(m, det_boxes[i], image.shape)
                      for i, m in enumerate(det_mask_specific)])
log("det_mask_specific", det_mask_specific)
log("det_masks", det_masks)"""

In [ ]:
#visualize.display_images(det_mask_specific[:4] * 255, cmap="Blues", interpolation="none")